# Demo 1: Zaawansowane Transformacje PySpark

**Temat:** Advanced PySpark Transformations  
**Czas trwania:** 60 minut  
**Typ:** Live coding demo + teoria

---

## 📋 Cele szkoleniowe

- Window Functions: partitionBy, orderBy, rowsBetween, rangeBetween
- Funkcje okienkowe: lag, lead, row_number, dense_rank, rank
- Rolling windows i agregacje ruchome
- Struktury złożone: explode, posexplode, sequence
- JSON processing: from_json, to_json, schema_of_json
- Funkcje datowe: date_trunc, date_add, add_months, last_day

---

## Inicjalizacja środowiska

In [ ]:
%run ../00_setup

---

## Część 1: Window Functions - podstawy

### Koncepcja Window Functions

Window Functions pozwalają na wykonywanie obliczeń na "oknach" (partycjach) danych bez aggregacji całego DataFrame.

**Kluczowe elementy:**
- `partitionBy()`: Podział danych na grupy
- `orderBy()`: Sortowanie w ramach partycji
- `rowsBetween()` / `rangeBetween()`: Definicja zakresu okna

**Zastosowania:**
- Ranking (row_number, rank, dense_rank)
- Porównania czasowe (lag, lead)
- Agregacje ruchome (rolling sum, moving average)
- Analiza trendów

In [ ]:
from pyspark.sql import Window
from pyspark.sql.functions import (
    col, row_number, rank, dense_rank, lag, lead,
    sum as _sum, avg, count, max as _max, min as _min,
    to_date, date_trunc, date_add, add_months, last_day,
    explode, posexplode, sequence, from_json, to_json, schema_of_json,
    current_timestamp, round as _round, lit
)
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType, DateType, ArrayType
import datetime

# Przygotowanie przykładowych danych zamówień
orders_data = [
    (1, 1, "2024-01-15", 150.0),
    (2, 2, "2024-01-16", 200.0),
    (3, 1, "2024-02-10", 300.0),
    (4, 3, "2024-02-12", 100.0),
    (5, 2, "2024-03-05", 450.0),
    (6, 1, "2024-03-15", 250.0),
    (7, 3, "2024-03-20", 180.0),
    (8, 2, "2024-04-01", 320.0),
    (9, 1, "2024-04-10", 400.0),
    (10, 3, "2024-04-15", 220.0),
]

orders_schema = StructType([
    StructField("order_id", IntegerType(), False),
    StructField("customer_id", IntegerType(), False),
    StructField("order_date", StringType(), False),
    StructField("amount", DoubleType(), False)
])

orders_df = spark.createDataFrame(orders_data, orders_schema)
orders_df = orders_df.withColumn("order_date", to_date(col("order_date")))

orders_df.display()

### Przykład 1: Ranking - row_number, rank, dense_rank

**Różnice:**
- `row_number()`: Unikalne numery (1, 2, 3, 4, ...)
- `rank()`: Z przerwami przy równych wartościach (1, 2, 2, 4, ...)
- `dense_rank()`: Bez przerw przy równych wartościach (1, 2, 2, 3, ...)

In [ ]:
# Ranking zamówień dla każdego klienta według kwoty (malejąco)
window_spec = Window.partitionBy("customer_id").orderBy(col("amount").desc())

orders_ranked = orders_df.withColumn("row_num", row_number().over(window_spec)) \
    .withColumn("rank", rank().over(window_spec)) \
    .withColumn("dense_rank", dense_rank().over(window_spec))

orders_ranked.orderBy("customer_id", "row_num").display()

### Przykład 2: Funkcje lag i lead

**lag()**: Wartość z poprzedniego wiersza  
**lead()**: Wartość z następnego wiersza

**Zastosowania:**
- Porównanie z poprzednim okresem
- Obliczenie zmian (deltas)
- Analiza sekwencji zdarzeń

In [ ]:
# Porównanie z poprzednim i następnym zamówieniem
window_spec_time = Window.partitionBy("customer_id").orderBy("order_date")

orders_lag_lead = orders_df \
    .withColumn("prev_amount", lag("amount", 1).over(window_spec_time)) \
    .withColumn("next_amount", lead("amount", 1).over(window_spec_time)) \
    .withColumn("amount_change", col("amount") - col("prev_amount")) \
    .withColumn("amount_change_pct", 
                _round((col("amount") - col("prev_amount")) / col("prev_amount") * 100, 2))

orders_lag_lead.orderBy("customer_id", "order_date").display()

---

## Część 2: Rolling Windows - agregacje ruchome

### rowsBetween vs rangeBetween

**rowsBetween(start, end)**: Zakres wierszy  
- `Window.unboundedPreceding`: Od początku partycji  
- `Window.unboundedFollowing`: Do końca partycji  
- `-1, 0`: Poprzedni wiersz i bieżący  
- `-2, 0`: Dwa poprzednie + bieżący (3-row window)

**rangeBetween(start, end)**: Zakres wartości (wymaga orderBy)

In [ ]:
# Rolling sum - suma krocząca (wszystkie poprzednie + bieżący)
window_cumulative = Window.partitionBy("customer_id") \
    .orderBy("order_date") \
    .rowsBetween(Window.unboundedPreceding, Window.currentRow)

orders_cumulative = orders_df \
    .withColumn("cumulative_sum", _sum("amount").over(window_cumulative)) \
    .withColumn("cumulative_count", count("order_id").over(window_cumulative)) \
    .withColumn("cumulative_avg", _round(avg("amount").over(window_cumulative), 2))

orders_cumulative.orderBy("customer_id", "order_date").display()

In [ ]:
# Moving average - średnia krocząca z 3 ostatnich zamówień
window_moving_3 = Window.partitionBy("customer_id") \
    .orderBy("order_date") \
    .rowsBetween(-2, 0)  # 2 poprzednie + bieżący = 3 wiersze

orders_moving_avg = orders_df \
    .withColumn("moving_avg_3", _round(avg("amount").over(window_moving_3), 2)) \
    .withColumn("moving_sum_3", _sum("amount").over(window_moving_3)) \
    .withColumn("moving_max_3", _max("amount").over(window_moving_3)) \
    .withColumn("moving_min_3", _min("amount").over(window_moving_3))

orders_moving_avg.orderBy("customer_id", "order_date").display()

---

## Część 3: Struktury złożone - Arrays

### explode i posexplode

**explode()**: Rozwija tablicę do osobnych wierszy  
**posexplode()**: Jak explode, ale z numerem pozycji

**Zastosowania:**
- Normalizacja danych zagnieżdżonych
- Analiza list (tagi, kategorie, produkty)
- Event tracking (sekwencje akcji)

In [ ]:
# Przykład: Produkty w zamówieniach (array)
orders_with_products_data = [
    (1, 1, "2024-01-15", ["Laptop", "Mouse", "Keyboard"]),
    (2, 2, "2024-01-16", ["Monitor", "Cable"]),
    (3, 1, "2024-02-10", ["Headphones"]),
    (4, 3, "2024-02-12", ["Tablet", "Case", "Stylus", "Charger"]),
]

orders_products_schema = StructType([
    StructField("order_id", IntegerType(), False),
    StructField("customer_id", IntegerType(), False),
    StructField("order_date", StringType(), False),
    StructField("products", ArrayType(StringType()), False)
])

orders_products_df = spark.createDataFrame(orders_with_products_data, orders_products_schema)

print("Dane oryginalne (z arrayami):")
orders_products_df.display()

In [ ]:
# explode() - rozwija tablicę do osobnych wierszy
orders_exploded = orders_products_df.select(
    "order_id",
    "customer_id",
    "order_date",
    explode("products").alias("product")
)

print("Dane po explode():")
orders_exploded.display()

In [ ]:
# posexplode() - rozwija tablicę z numerem pozycji
orders_posexploded = orders_products_df.select(
    "order_id",
    "customer_id",
    "order_date",
    posexplode("products").alias("position", "product")
)

print("Dane po posexplode() - z numerem pozycji:")
orders_posexploded.display()

### sequence() - generowanie sekwencji

**sequence(start, stop, step)**: Generuje tablicę wartości

**Zastosowania:**
- Generowanie zakresów dat
- Tworzenie series czasowych
- Wypełnianie braków w danych

In [ ]:
# Przykład: Generowanie sekwencji dni między datami
from pyspark.sql.functions import expr

date_ranges_data = [
    ("2024-01-01", "2024-01-05"),
    ("2024-02-01", "2024-02-03"),
]

date_ranges_df = spark.createDataFrame(date_ranges_data, ["start_date", "end_date"]) \
    .withColumn("start_date", to_date(col("start_date"))) \
    .withColumn("end_date", to_date(col("end_date")))

# Generuj sekwencję dni
date_sequence = date_ranges_df.withColumn(
    "date_array",
    expr("sequence(start_date, end_date, interval 1 day)")
)

print("Sekwencja dat jako array:")
date_sequence.display()

# Rozwij do osobnych wierszy
print("\nSekwencja dat po explode:")
date_sequence.select(
    "start_date",
    "end_date",
    explode("date_array").alias("date")
).display()

---

## Część 4: JSON Processing

### from_json, to_json, schema_of_json

**from_json()**: Parsowanie JSON string → struct/array  
**to_json()**: Konwersja struct/array → JSON string  
**schema_of_json()**: Automatyczne wykrycie schematu JSON

**Zastosowania:**
- Parsowanie API responses
- Event tracking (nested JSON events)
- Log processing

In [ ]:
# Przykład: Parsowanie JSON payload
json_data = [
    (1, '{"user_id": 101, "action": "click", "metadata": {"page": "home", "duration": 30}}'),
    (2, '{"user_id": 102, "action": "purchase", "metadata": {"page": "checkout", "duration": 120}}'),
    (3, '{"user_id": 101, "action": "view", "metadata": {"page": "product", "duration": 45}}'),
]

json_df = spark.createDataFrame(json_data, ["event_id", "json_payload"])

print("Dane oryginalne (JSON jako string):")
json_df.display()

In [ ]:
# Automatyczne wykrycie schematu JSON
json_schema = schema_of_json(lit(json_data[0][1]))
print(f"Wykryty schemat: {json_schema}")

# Parsowanie JSON
json_parsed = json_df.withColumn("parsed_data", from_json(col("json_payload"), json_schema))

print("\nDane po parsowaniu JSON:")
json_parsed.display()

In [ ]:
# Wyciąganie pól z zagnieżdżonego JSON
json_flattened = json_parsed.select(
    "event_id",
    col("parsed_data.user_id").alias("user_id"),
    col("parsed_data.action").alias("action"),
    col("parsed_data.metadata.page").alias("page"),
    col("parsed_data.metadata.duration").alias("duration")
)

print("Dane po spłaszczeniu (flattening):")
json_flattened.display()

---

## Część 5: Funkcje datowe i czasowe

### Kluczowe funkcje

**date_trunc()**: Obcięcie do granicy (rok, miesiąc, dzień, godzina)  
**date_add()**: Dodawanie dni  
**add_months()**: Dodawanie miesięcy  
**last_day()**: Ostatni dzień miesiąca  
**datediff()**: Różnica w dniach  
**months_between()**: Różnica w miesiącach

**Zastosowania:**
- Agregacje temporalne (daily, monthly, yearly)
- Analiza cohort
- Retention analysis
- Forecast horizons

In [ ]:
from pyspark.sql.functions import datediff, months_between, year, month, dayofweek, quarter

# Przykład: Analiza temporalna zamówień
orders_temporal = orders_df \
    .withColumn("year", year("order_date")) \
    .withColumn("month", month("order_date")) \
    .withColumn("quarter", quarter("order_date")) \
    .withColumn("day_of_week", dayofweek("order_date")) \
    .withColumn("month_start", date_trunc("month", "order_date")) \
    .withColumn("month_end", last_day("order_date")) \
    .withColumn("next_month_start", date_add(last_day("order_date"), 1))

orders_temporal.display()

In [ ]:
# Przykład: Obliczanie okresów między zamówieniami
window_date = Window.partitionBy("customer_id").orderBy("order_date")

orders_periods = orders_df \
    .withColumn("prev_order_date", lag("order_date", 1).over(window_date)) \
    .withColumn("days_since_last_order", 
                datediff(col("order_date"), col("prev_order_date"))) \
    .withColumn("months_since_last_order", 
                _round(months_between(col("order_date"), col("prev_order_date")), 2))

orders_periods.orderBy("customer_id", "order_date").display()

---

## Część 6: Praktyczny przykład - Customer Behavior Analysis

### Zadanie: Analiza zachowań klientów

Wykorzystamy wszystkie poznane techniki do kompleksowej analizy:
1. Ranking zamówień dla każdego klienta
2. Porównanie z poprzednim zamówieniem (lag)
3. Średnia ruchoma wydatków
4. Segmentacja temporalna

In [ ]:
# Kompleksowa analiza zachowań klientów
window_customer_time = Window.partitionBy("customer_id").orderBy("order_date")
window_customer_amount = Window.partitionBy("customer_id").orderBy(col("amount").desc())
window_moving_avg = Window.partitionBy("customer_id").orderBy("order_date").rowsBetween(-2, 0)

customer_behavior = orders_df \
    .withColumn("order_rank", row_number().over(window_customer_amount)) \
    .withColumn("order_sequence", row_number().over(window_customer_time)) \
    .withColumn("prev_amount", lag("amount", 1).over(window_customer_time)) \
    .withColumn("amount_change", col("amount") - col("prev_amount")) \
    .withColumn("moving_avg_3", _round(avg("amount").over(window_moving_avg), 2)) \
    .withColumn("cumulative_spent", _sum("amount").over(
        Window.partitionBy("customer_id").orderBy("order_date")
        .rowsBetween(Window.unboundedPreceding, Window.currentRow)
    )) \
    .withColumn("month", date_trunc("month", "order_date"))

customer_behavior.orderBy("customer_id", "order_date").display()

---

## Podsumowanie

### ✅ Omówione zagadnienia

1. **Window Functions**
   - partitionBy, orderBy
   - row_number, rank, dense_rank
   - lag, lead

2. **Rolling Windows**
   - rowsBetween / rangeBetween
   - Cumulative aggregations
   - Moving averages

3. **Struktury złożone**
   - explode / posexplode
   - sequence()

4. **JSON Processing**
   - from_json, to_json
   - schema_of_json
   - Flattening nested JSON

5. **Funkcje datowe**
   - date_trunc, date_add, add_months, last_day
   - datediff, months_between
   - Temporal aggregations

---

### 🎯 Best Practices

1. **Window Functions**: Używaj partitionBy dla efektywności
2. **Rolling Windows**: Wybierz odpowiedni zakres (rows vs range)
3. **explode**: Uważaj na performance przy dużych arrayach
4. **JSON**: Wykorzystuj schema_of_json dla automatycznego wykrycia struktury
5. **Temporal**: Standaryzuj strefy czasowe przed analizą

---